### Geçmiş Maçlar Fonksiyonu

In [6]:
import http.client
import json
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


def FullData(date):

    conn = http.client.HTTPSConnection('www.sofascore.com')
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'tr,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6,es;q=0.5',
        'cache-control': 'max-age=0',
        'cookie': '_lr_retry_request=true; _lr_env_src_ats=false; __browsiSessionID=b7b86200-3306-4ccd-8fb8-5a17eb6d0ec1&false&SEARCH&tr&desktop-4.28.136&false; __browsiUID=6a56a669-d1a3-41a9-a34a-14881cc4b5d9; _ga=GA1.1.1496961110.1725713349; __qca=P0-2010655032-1725713348258; __gads=ID=0aab5560fdd047e5:T=1725713346:RT=1725713346:S=ALNI_MbCdw13CMcl6SndpWG3M_mEVzEfcA; __gpi=UID=00000ede7a0414d9:T=1725713346:RT=1725713346:S=ALNI_MbkBEvZZ3oc7xmvGVQekfYohbe2Sw; __eoi=ID=ff1ddb78af6fba8e:T=1725713346:RT=1725713346:S=AA-AfjaIrZWOkM_AaPL3N516P5pC; gcid_first=69f61960-e83b-46b2-84b2-948aa83451ac; FCNEC=%5B%5B%22AKsRol-6Agw9FKoOehBMSItXqsBvnFkb5cxleV1LSlXVjdk2YnG4jmoLLlK8q1XasCO2rxC3Rxm9pFMTlhAf5-uG35unf-7RT3SrMHTKEcbJ0W8g1DRWQCHkGtXur2--Ib97pqXYBPnKMn-UDYjNY6cp7TT4Gm687w%3D%3D%22%5D%5D; _ga_HNQ9P9MGZR=GS1.1.1725713348.1.1.1725713495.24.0.0',
        'if-none-match': 'W/"6256a43754"',
        'priority': 'u=0, i',
        'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    }
    conn.request('GET', '/api/v1/sport/football/scheduled-events/'+str(date), headers=headers)
    response = conn.getresponse()

    data = json.loads(response.read())["events"]
    data1 = pd.json_normalize(data, sep='_')
    columns_to_select = [
        'customId', 'id', 'startTimestamp', 'tournament_name', 'tournament_category_name',
        'season_name', 'season_year', 'tournament_uniqueTournament_id', 'season_id', 'roundInfo_round',
        'status_type', 'homeTeam_name', 'homeTeam_nameCode', 'homeTeam_id', 'awayTeam_name',
        'awayTeam_nameCode', 'awayTeam_id', 'homeScore_display', 'homeScore_period1',
        'homeScore_period2', 'awayScore_display', 'awayScore_period1', 'awayScore_period2'
    ]

    for column in columns_to_select:
        if column not in data1.columns:
            data1[column] = ""  
            
    df_selected = data1[columns_to_select]
    df_selected.insert(0, "Tarih", date)
    df_selected['startTimestamp'] = pd.to_numeric(df_selected['startTimestamp'], errors='coerce')
    df_selected = df_selected.dropna(subset=['startTimestamp']) 
    df_selected['startTimestamp'] = df_selected['startTimestamp'].astype(int)
    df_selected.loc[:, 'startTimestamp'] = pd.to_datetime(df_selected['startTimestamp'], unit='s')
    df_selected.loc[:, 'startTimestamp'] = df_selected['startTimestamp'] + pd.Timedelta(hours=3)
    return df_selected

df = FullData("2024-09-28")
df.to_excel("GununMaclari.xlsx")

### Gol Dakika Fonksiyonu

In [5]:
import http.client
import pandas as pd
import json

def GoalMinute(match_id):
    conn = http.client.HTTPSConnection('www.sofascore.com')
    headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'accept-language': 'tr,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6,es;q=0.5',
            'cache-control': 'max-age=0',
            'cookie': '_lr_retry_request=true; _lr_env_src_ats=false; __browsiSessionID=b7b86200-3306-4ccd-8fb8-5a17eb6d0ec1&false&SEARCH&tr&desktop-4.28.136&false; __browsiUID=6a56a669-d1a3-41a9-a34a-14881cc4b5d9; _ga=GA1.1.1496961110.1725713349; __qca=P0-2010655032-1725713348258; __gads=ID=0aab5560fdd047e5:T=1725713346:RT=1725713346:S=ALNI_MbCdw13CMcl6SndpWG3M_mEVzEfcA; __gpi=UID=00000ede7a0414d9:T=1725713346:RT=1725713346:S=ALNI_MbkBEvZZ3oc7xmvGVQekfYohbe2Sw; __eoi=ID=ff1ddb78af6fba8e:T=1725713346:RT=1725713346:S=AA-AfjaIrZWOkM_AaPL3N516P5pC; gcid_first=69f61960-e83b-46b2-84b2-948aa83451ac; FCNEC=%5B%5B%22AKsRol-6Agw9FKoOehBMSItXqsBvnFkb5cxleV1LSlXVjdk2YnG4jmoLLlK8q1XasCO2rxC3Rxm9pFMTlhAf5-uG35unf-7RT3SrMHTKEcbJ0W8g1DRWQCHkGtXur2--Ib97pqXYBPnKMn-UDYjNY6cp7TT4Gm687w%3D%3D%22%5D%5D; _ga_HNQ9P9MGZR=GS1.1.1725713348.1.1.1725713495.24.0.0',
            'if-none-match': 'W/"6256a43754"',
            'priority': 'u=0, i',
            'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'none',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
        }
    conn.request('GET', 'https://www.sofascore.com/api/v1/event/'+str(match_id)+'/incidents', headers=headers)
    
    response = conn.getresponse()
    
    data = json.loads(response.read())["incidents"]
    data1 = pd.json_normalize(data, sep='_')
    data1["match_id"] = match_id
    columns_to_select = [
         "time","incidentType","isHome","incidentClass","player_name","assist1_name","match_id"
    ]

    for column in columns_to_select:
        if column not in data1.columns:
            data1[column] = ""  

    df_selected = data1[columns_to_select]
    dfgoal = df_selected[df_selected["incidentType"]=="goal"]
    
    return dfgoal 

df=GoalMinute(12057912)
df

,time,incidentType,isHome,incidentClass,player_name,assist1_name,match_id
14,52,goal,True,penalty,Kevin De Bruyne,NaN,12057912
15,48,goal,True,regular,Youri Tielemans,Loïs Openda,12057912
18,36,goal,False,ownGoal,Timothy Castagne,NaN,12057912
20,21,goal,True,regular,Kevin De Bruyne,Jérémy Doku,12057912


### Maç Baskisi Data

In [6]:
import http.client
import json
import pandas as pd

def MacBaski(match_id):


    conn = http.client.HTTPSConnection('www.sofascore.com')
    headers = {
                'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
                'accept-language': 'tr,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6,es;q=0.5',
                'cache-control': 'max-age=0',
                'cookie': '_lr_retry_request=true; _lr_env_src_ats=false; __browsiSessionID=b7b86200-3306-4ccd-8fb8-5a17eb6d0ec1&false&SEARCH&tr&desktop-4.28.136&false; __browsiUID=6a56a669-d1a3-41a9-a34a-14881cc4b5d9; _ga=GA1.1.1496961110.1725713349; __qca=P0-2010655032-1725713348258; __gads=ID=0aab5560fdd047e5:T=1725713346:RT=1725713346:S=ALNI_MbCdw13CMcl6SndpWG3M_mEVzEfcA; __gpi=UID=00000ede7a0414d9:T=1725713346:RT=1725713346:S=ALNI_MbkBEvZZ3oc7xmvGVQekfYohbe2Sw; __eoi=ID=ff1ddb78af6fba8e:T=1725713346:RT=1725713346:S=AA-AfjaIrZWOkM_AaPL3N516P5pC; gcid_first=69f61960-e83b-46b2-84b2-948aa83451ac; FCNEC=%5B%5B%22AKsRol-6Agw9FKoOehBMSItXqsBvnFkb5cxleV1LSlXVjdk2YnG4jmoLLlK8q1XasCO2rxC3Rxm9pFMTlhAf5-uG35unf-7RT3SrMHTKEcbJ0W8g1DRWQCHkGtXur2--Ib97pqXYBPnKMn-UDYjNY6cp7TT4Gm687w%3D%3D%22%5D%5D; _ga_HNQ9P9MGZR=GS1.1.1725713348.1.1.1725713495.24.0.0',
                'if-none-match': 'W/"6256a43754"',
                'priority': 'u=0, i',
                'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': '"Windows"',
                'sec-fetch-dest': 'document',
                'sec-fetch-mode': 'navigate',
                'sec-fetch-site': 'none',
                'sec-fetch-user': '?1',
                'upgrade-insecure-requests': '1',
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
            }
    conn.request('GET', '/api/v1/event/'+str(match_id)+'/graph', headers=headers)
    response = conn.getresponse()
    
    data = pd.DataFrame(json.loads(response.read())["graphPoints"])
    data["match_id"]=match_id
    
    return data
    
df = MacBaski(12057912)
df


,minute,value,match_id
0,1.0,-9,12057912
1,2.0,-31,12057912
2,3.0,-21,12057912
3,4.0,-3,12057912
4,5.0,23,12057912
...,...,...,...
87,87.0,21,12057912
88,88.0,7,12057912
89,89.0,4,12057912
90,90.0,4,12057912


In [7]:
import http.client
import json
import pandas as pd

def extract_player_info(player_data, team_type):
    # Temel istatistikler (BS)
    player_info = {
        
        'bs_team': team_type,
        'bs_name': player_data['player']['name'],
        'bs_shortName': player_data['player']['shortName'],
        'bs_position': player_data['player']['position'],
        'bs_jerseyNumber': player_data['player']['jerseyNumber'],
        'bs_country': player_data['player']['country']['name'],
        'bs_userCount': player_data['player']['userCount'],
        'bs_marketValueCurrency': player_data['player']['marketValueCurrency'],
        'bs_minutesPlayed': player_data['statistics'].get('minutesPlayed', 0),
        'bs_rating': player_data['statistics'].get('rating', 0)
    }

    # Atak istatistikleri (AT)
    attacking_stats = {
        'at_totalPass': player_data['statistics'].get('totalPass', 0),
        'at_accuratePass': player_data['statistics'].get('accuratePass', 0),
        'at_goalAssist': player_data['statistics'].get('goalAssist', 0),
        'at_shotOffTarget': player_data['statistics'].get('shotOffTarget', 0),
        'at_onTargetScoringAttempt': player_data['statistics'].get('onTargetScoringAttempt', 0),
        'at_bigChanceMissed': player_data['statistics'].get('bigChanceMissed', 0),
        'at_expectedGoals': player_data['statistics'].get('expectedGoals', 0),
        'at_expectedAssists': player_data['statistics'].get('expectedAssists', 0),
        'at_keyPass': player_data['statistics'].get('keyPass', 0),
        'at_totalLongBalls': player_data['statistics'].get('totalLongBalls', 0),
        'at_accurateLongBalls': player_data['statistics'].get('accurateLongBalls', 0),
        'at_touches': player_data['statistics'].get('touches', 0),
        'at_possessionLostCtrl': player_data['statistics'].get('possessionLostCtrl', 0)
    }

    # Defansif istatistikler (DE)
    defensive_stats = {
        'de_aerialWon': player_data['statistics'].get('aerialWon', 0),
        'de_duelLost': player_data['statistics'].get('duelLost', 0),
        'de_duelWon': player_data['statistics'].get('duelWon', 0),
        'de_challengeLost': player_data['statistics'].get('challengeLost', 0),
        'de_dispossessed': player_data['statistics'].get('dispossessed', 0),
        'de_totalContest': player_data['statistics'].get('totalContest', 0),
        'de_wonContest': player_data['statistics'].get('wonContest', 0),
        'de_outfielderBlock': player_data['statistics'].get('outfielderBlock', 0),
        'de_totalTackle': player_data['statistics'].get('totalTackle', 0),
        'de_fouls': player_data['statistics'].get('fouls', 0),
        'de_interceptionWon': player_data['statistics'].get('interceptionWon', 0),
        'de_totalClearance': player_data['statistics'].get('totalClearance', 0)
    }

    # Kaleci istatistikleri (GO)
    goalkeeper_stats = {
        'go_saves': player_data['statistics'].get('saves', 0),
        'go_goalsPrevented': player_data['statistics'].get('goalsPrevented', 0),
        'go_penaltySave': player_data['statistics'].get('penaltySave', 0)
    }

    # Tüm istatistikleri birleştiriyoruz
    player_info.update(attacking_stats)
    player_info.update(defensive_stats)
    player_info.update(goalkeeper_stats)
    
    
    return player_info
    
def LineUp(match_id):


    conn = http.client.HTTPSConnection('www.sofascore.com')
    headers = {
                'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
                'accept-language': 'tr,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6,es;q=0.5',
                'cache-control': 'max-age=0',
                'cookie': '_lr_retry_request=true; _lr_env_src_ats=false; __browsiSessionID=b7b86200-3306-4ccd-8fb8-5a17eb6d0ec1&false&SEARCH&tr&desktop-4.28.136&false; __browsiUID=6a56a669-d1a3-41a9-a34a-14881cc4b5d9; _ga=GA1.1.1496961110.1725713349; __qca=P0-2010655032-1725713348258; __gads=ID=0aab5560fdd047e5:T=1725713346:RT=1725713346:S=ALNI_MbCdw13CMcl6SndpWG3M_mEVzEfcA; __gpi=UID=00000ede7a0414d9:T=1725713346:RT=1725713346:S=ALNI_MbkBEvZZ3oc7xmvGVQekfYohbe2Sw; __eoi=ID=ff1ddb78af6fba8e:T=1725713346:RT=1725713346:S=AA-AfjaIrZWOkM_AaPL3N516P5pC; gcid_first=69f61960-e83b-46b2-84b2-948aa83451ac; FCNEC=%5B%5B%22AKsRol-6Agw9FKoOehBMSItXqsBvnFkb5cxleV1LSlXVjdk2YnG4jmoLLlK8q1XasCO2rxC3Rxm9pFMTlhAf5-uG35unf-7RT3SrMHTKEcbJ0W8g1DRWQCHkGtXur2--Ib97pqXYBPnKMn-UDYjNY6cp7TT4Gm687w%3D%3D%22%5D%5D; _ga_HNQ9P9MGZR=GS1.1.1725713348.1.1.1725713495.24.0.0',
                'if-none-match': 'W/"6256a43754"',
                'priority': 'u=0, i',
                'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': '"Windows"',
                'sec-fetch-dest': 'document',
                'sec-fetch-mode': 'navigate',
                'sec-fetch-site': 'none',
                'sec-fetch-user': '?1',
                'upgrade-insecure-requests': '1',
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
            }
    conn.request('GET', '/api/v1/event/'+str(match_id)+'/lineups', headers=headers)
    response = conn.getresponse()
    data = json.loads(response.read())

    players_list = []

    for player in data['home']['players']:
        players_list.append(extract_player_info(player, 'home'))
    
    for player in data['away']['players']:
        players_list.append(extract_player_info(player, 'away'))
    
    # DataFrame'e çevirme
    df = pd.DataFrame(players_list)
    df["home_formation"] = data["home"]["formation"]
    df["away_formation"] = data["away"]["formation"]
    df["match_id"] = match_id
   
    
    return df

data = LineUp(12436927)

data

,bs_team,bs_name,bs_shortName,bs_position,bs_jerseyNumber,bs_country,bs_userCount,bs_marketValueCurrency,bs_minutesPlayed,bs_rating,...,de_totalTackle,de_fouls,de_interceptionWon,de_totalClearance,go_saves,go_goalsPrevented,go_penaltySave,home_formation,away_formation,match_id
0,home,Alphonse Aréola,A. Aréola,G,23,France,4104,EUR,45,6.9,...,0,0,0,0,3,-0.3302,0,4-2-3-1,3-2-4-1,12436927
1,home,Aaron Wan-Bissaka,A. Wan-Bissaka,D,29,England,14284,EUR,90,6.7,...,2,1,2,2,0,0.0000,0,4-2-3-1,3-2-4-1,12436927
2,home,Konstantinos Mavropanos,K. Mavropanos,D,15,Greece,2708,EUR,90,6.3,...,0,0,0,4,0,0.0000,0,4-2-3-1,3-2-4-1,12436927
3,home,Max Kilman,M. Kilman,D,26,England,1428,EUR,90,6.3,...,0,1,0,4,0,0.0000,0,4-2-3-1,3-2-4-1,12436927
4,home,Emerson Palmieri,Emerson,D,33,Italy,2631,EUR,71,6.3,...,2,1,0,3,0,0.0000,0,4-2-3-1,3-2-4-1,12436927
5,home,Guido Rodríguez,G. Rodríguez,M,24,Argentina,6575,EUR,90,6.9,...,0,2,3,2,0,0.0000,0,4-2-3-1,3-2-4-1,12436927
6,home,Edson Álvarez,E. Álvarez,M,19,Mexico,8633,EUR,77,6.7,...,1,1,0,2,0,0.0000,0,4-2-3-1,3-2-4-1,12436927
7,home,Jarrod Bowen,J. Bowen,M,20,England,8665,EUR,86,6.8,...,2,0,0,1,0,0.0000,0,4-2-3-1,3-2-4-1,12436927
8,home,Lucas Paquetá,L. Paquetá,M,10,Brazil,58402,EUR,90,6.5,...,2,2,1,0,0,0.0000,0,4-2-3-1,3-2-4-1,12436927
9,home,Mohammed Kudus,M. Kudus,M,14,Ghana,75749,EUR,90,6.6,...,2,0,0,0,0,0.0000,0,4-2-3-1,3-2-4-1,12436927



### Maç İstatistikleri

In [8]:
import http.client
import json
import pandas as pd

def MacStats(match_id):


    conn = http.client.HTTPSConnection('www.sofascore.com')
    headers = {
                'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
                'accept-language': 'tr,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6,es;q=0.5',
                'cache-control': 'max-age=0',
                'cookie': '_lr_retry_request=true; _lr_env_src_ats=false; __browsiSessionID=b7b86200-3306-4ccd-8fb8-5a17eb6d0ec1&false&SEARCH&tr&desktop-4.28.136&false; __browsiUID=6a56a669-d1a3-41a9-a34a-14881cc4b5d9; _ga=GA1.1.1496961110.1725713349; __qca=P0-2010655032-1725713348258; __gads=ID=0aab5560fdd047e5:T=1725713346:RT=1725713346:S=ALNI_MbCdw13CMcl6SndpWG3M_mEVzEfcA; __gpi=UID=00000ede7a0414d9:T=1725713346:RT=1725713346:S=ALNI_MbkBEvZZ3oc7xmvGVQekfYohbe2Sw; __eoi=ID=ff1ddb78af6fba8e:T=1725713346:RT=1725713346:S=AA-AfjaIrZWOkM_AaPL3N516P5pC; gcid_first=69f61960-e83b-46b2-84b2-948aa83451ac; FCNEC=%5B%5B%22AKsRol-6Agw9FKoOehBMSItXqsBvnFkb5cxleV1LSlXVjdk2YnG4jmoLLlK8q1XasCO2rxC3Rxm9pFMTlhAf5-uG35unf-7RT3SrMHTKEcbJ0W8g1DRWQCHkGtXur2--Ib97pqXYBPnKMn-UDYjNY6cp7TT4Gm687w%3D%3D%22%5D%5D; _ga_HNQ9P9MGZR=GS1.1.1725713348.1.1.1725713495.24.0.0',
                'if-none-match': 'W/"6256a43754"',
                'priority': 'u=0, i',
                'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': '"Windows"',
                'sec-fetch-dest': 'document',
                'sec-fetch-mode': 'navigate',
                'sec-fetch-site': 'none',
                'sec-fetch-user': '?1',
                'upgrade-insecure-requests': '1',
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
            }
    
    conn.request('GET', '/api/v1/event/'+str(match_id)+'/statistics', headers=headers)
    response = conn.getresponse()

    data = json.loads(response.read())["statistics"]
    all_data = pd.DataFrame()
  
    for period_data in data:
        period = period_data['period']
        for group in period_data['groups']:
            group_name = group['groupName']
            stats = group['statisticsItems']
            
            # Veriyi DataFrame'e çevir
            df = pd.json_normalize(stats)
            
            # Ekstra sütunlar ekle: period ve groupName
            df['period'] = period
            df['groupName'] = group_name
            
            # Sonuçları ana DataFrame'e ekle
            all_data = pd.concat([all_data, df], ignore_index=True)
            all_data["match_id"] = match_id
    
    return all_data
      

data = MacStats(12436927)
data

,name,home,away,compareCode,statisticsType,valueType,homeValue,awayValue,renderType,key,period,groupName,match_id,homeTotal,awayTotal
0,Ball possession,32%,68%,2,positive,event,32.00,68.0,2,ballPossession,ALL,Match overview,12436927,NaN,NaN
1,Expected goals,0.73,3.00,2,positive,event,0.73,3.0,1,expectedGoals,ALL,Match overview,12436927,NaN,NaN
2,Big chances,1,5,2,positive,event,1.00,5.0,1,bigChanceCreated,ALL,Match overview,12436927,NaN,NaN
3,Total shots,10,23,2,positive,event,10.00,23.0,1,totalShotsOnGoal,ALL,Match overview,12436927,NaN,NaN
4,Goalkeeper saves,5,2,1,positive,event,5.00,2.0,1,goalkeeperSaves,ALL,Match overview,12436927,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,Interceptions,2,3,2,positive,event,2.00,3.0,1,interceptionWon,2ND,Defending,12436927,NaN,NaN
119,Recoveries,19,19,3,positive,event,19.00,19.0,1,ballRecovery,2ND,Defending,12436927,NaN,NaN
120,Clearances,6,11,2,positive,event,6.00,11.0,1,totalClearance,2ND,Defending,12436927,NaN,NaN
121,Total saves,2,1,1,positive,event,2.00,1.0,1,goalkeeperSaves,2ND,Goalkeeping,12436927,NaN,NaN


# Lig Verileri

### Puan Durumu Ev

In [9]:
import http.client
import json
import pandas as pd

def LigPuanDurum(tournament,season,tur):

    conn = http.client.HTTPSConnection('www.sofascore.com')
    headers = {
                    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
                    'accept-language': 'tr,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6,es;q=0.5',
                    'cache-control': 'max-age=0',
                    'cookie': '_lr_retry_request=true; _lr_env_src_ats=false; __browsiSessionID=b7b86200-3306-4ccd-8fb8-5a17eb6d0ec1&false&SEARCH&tr&desktop-4.28.136&false; __browsiUID=6a56a669-d1a3-41a9-a34a-14881cc4b5d9; _ga=GA1.1.1496961110.1725713349; __qca=P0-2010655032-1725713348258; __gads=ID=0aab5560fdd047e5:T=1725713346:RT=1725713346:S=ALNI_MbCdw13CMcl6SndpWG3M_mEVzEfcA; __gpi=UID=00000ede7a0414d9:T=1725713346:RT=1725713346:S=ALNI_MbkBEvZZ3oc7xmvGVQekfYohbe2Sw; __eoi=ID=ff1ddb78af6fba8e:T=1725713346:RT=1725713346:S=AA-AfjaIrZWOkM_AaPL3N516P5pC; gcid_first=69f61960-e83b-46b2-84b2-948aa83451ac; FCNEC=%5B%5B%22AKsRol-6Agw9FKoOehBMSItXqsBvnFkb5cxleV1LSlXVjdk2YnG4jmoLLlK8q1XasCO2rxC3Rxm9pFMTlhAf5-uG35unf-7RT3SrMHTKEcbJ0W8g1DRWQCHkGtXur2--Ib97pqXYBPnKMn-UDYjNY6cp7TT4Gm687w%3D%3D%22%5D%5D; _ga_HNQ9P9MGZR=GS1.1.1725713348.1.1.1725713495.24.0.0',
                    'if-none-match': 'W/"6256a43754"',
                    'priority': 'u=0, i',
                    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
                    'sec-ch-ua-mobile': '?0',
                    'sec-ch-ua-platform': '"Windows"',
                    'sec-fetch-dest': 'document',
                    'sec-fetch-mode': 'navigate',
                    'sec-fetch-site': 'none',
                    'sec-fetch-user': '?1',
                    'upgrade-insecure-requests': '1',
                    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
                }
    conn.request(
        'GET',
        '/api/v1/unique-tournament/'+str(tournament)+'/season/'+str(season)+'/standings/'+str(tur),
        headers=headers
    )
    response = conn.getresponse()
    data = json.loads(response.read())["standings"]
    rows = data[0]['rows']
    
    teams_data = []
    for row in rows:
        team_info = {
            'Team': row['team']['name'],
            'Position': row['position'],
            'Matches': row['matches'],
            'Wins': row['wins'],
            'Draws': row['draws'],
            'Losses': row['losses'],
            'Goals For': row['scoresFor'],
            'Goals Against': row['scoresAgainst'],
            'Points': row['points']
        }
        teams_data.append(team_info)
    
    # DataFrame'e dönüştürme
    df = pd.DataFrame(teams_data)
    df["tournament"] = tournament
    df["season"]=season
    df["tur"]=tur
    return df
    
def FullPuanDur(tournament,season):
    datatotal = LigPuanDurum(tournament,season,"total")
    datahome= LigPuanDurum(tournament,season,"home")
    dataaway= LigPuanDurum(tournament,season,"away")
    fulldata = pd.concat([datatotal,datahome,dataaway])
    return fulldata

datam =FullPuanDur(52,63814)

datam

,Team,Position,Matches,Wins,Draws,Losses,Goals For,Goals Against,Points,tournament,season,tur
0,Fenerbahçe,1,4,3,1,0,11,2,10,52,63814,total
1,Galatasaray,2,3,3,0,0,9,3,9,52,63814,total
2,Beşiktaş,3,3,3,0,0,8,2,9,52,63814,total
3,Eyüpspor,4,4,2,2,0,6,2,8,52,63814,total
4,Başakşehir FK,5,3,2,1,0,10,5,7,52,63814,total
5,Göztepe,6,4,1,3,0,5,3,6,52,63814,total
6,Yılport Samsunspor,7,3,2,0,1,4,2,6,52,63814,total
7,Kasımpaşa,8,4,1,2,1,6,6,5,52,63814,total
8,Konyaspor,9,4,1,1,2,5,7,4,52,63814,total
9,Sivasspor,10,4,1,1,2,2,4,4,52,63814,total
